## Analyzing Aggravated Burglaries in Davidson County

### Part 1 - Data Gathering using APIs

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents () that were reported during the nine month period from January 1, 2022 through September 30, 2022. (Hint: Check out the API Docs to see how to narrow down the response to just the desired results).

In [1]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import re
import json

Reading the API, I figured out I need the NIBRS code (https://ucr.fbi.gov/nibrs/2011/resources/nibrs-offense-codes).
For burglary it is 220 and checking it on the webpage:


It looks like this is what I'm looking for and the 'offense description' isn't always consistent. Will have to check which ones have 'aggravated' in there:
https://data.nashville.gov/resource/2u6v-ujjs.json?offense_nibrs=220

The code below just connects to the API and gets the requested type (burglary/breaking and entering)

In [2]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json'

params = {
    '$where' : "incident_reported between '2022-01-01T00:00:00' and '2022-09-30T23:59:59'",
    'offense_nibrs' : 220,
    '$limit' : 5000
}

response = requests.get(endpoint, params = params)
response


<Response [200]>

In [3]:
len(response.json())

3528

In [4]:
incidents_df = pd.DataFrame(response.json())

In [5]:
# I used this before I figured out how to do it with regex
# incidents_df[incidents_df['offense_description'].str.upper().str.find('AGGRAVATED') != -1]

the below code is to get all the burglaries that contain at least agg (shorthand for AGGRAVATED). Comparing it to the output with my previous code I found out that the word AGGRAVATED was never actually abbreviated in the file

In [6]:
incidents_df = incidents_df[incidents_df['offense_description'].str.match(r'.+[aA][gG][gG].+') == True]

In [7]:
incidents_df['offense_description'].unique()

array(['BURGLARY- AGGRAVATED', 'BURGLARY - AGGRAVATED',
       'Burglary - Aggravated - Acting in Concert',
       'BURGLARY - ESPECIALLY AGGRAVATED',
       'BURGLARY - AGGRAVATED - ACTING IN CONCERT'], dtype=object)

2. Using the 2020 American Community Survey API, obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37.

First off, loading my key for the census data API!

In [8]:
with open('../keys.json') as fi:
    credentials = json.load(fi)

In [9]:
api_key = credentials['api_key']

> note: the documentation for how to construct the API request (for detailed tables) is found here: https://api.census.gov/data/2020/acs/acs5/examples.html

In [10]:
endpoint = f'https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key={api_key}'


response = requests.get(endpoint)
response

<Response [200]>

In [11]:
pop_df = pd.DataFrame(response.json())

In [13]:
pop_df = (
    pop_df
    .rename(columns=pop_df.iloc[0])
    .drop(pop_df.index[0])
    .rename(columns={'B01001_001E':'population'})
)

Similar but different, Subject Tables now. 

In [14]:
endpoint = f'https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,S1901_C01_012E&for=tract:*&in=state:47&in=county:037&key={api_key}'


response = requests.get(endpoint)
response

<Response [200]>

In [15]:
med_income_df = pd.DataFrame(response.json())

In [16]:
med_income_df = (
    med_income_df
    .rename(columns=med_income_df.iloc[0])
    .drop(med_income_df.index[0])
    .rename(columns={'S1901_C01_012E':'median_income'})
)

Did a .head() and .count() command to make sure both tables have the same length (as they should, 174); after the merge below the number was the same (as it of course should).

In [20]:
pop_med_income_df = pd.merge(pop_df, med_income_df, "inner", on=['NAME', 'state', 'county', 'tract'])